# Bokeh Exercises

*The "User Guide" linked on this page is a very good resource :* https://bokeh.pydata.org/en/latest/


## 1. Create a single Bokeh plot of some data 

*Include the standard pan, wheel_zoom, box_zoom, reset, save, box_select, and lasso_select tools, but no other special widgets*

## 2. Create a single Bokeh plot of some data with some widget

*Include the standard pan, wheel_zoom, box_zoom, reset, save, box_select, and lasso_select tools, and also a widget (e.g., slider, dropdown selection, etc.)  that controls some aspect of the plot*

In [132]:
### Todo ###
############
    # Different colors for predicted d states? rows on ptable?
    # Drop-down for plotting different axes?
    # Make glyphs letters

import numpy as np
import pandas as pd
from bokeh.plotting import *
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CDSView, CustomJS, Slider, Button, DataTable, TableColumn, NumberFormatter
from bokeh.models.filters import Filter, GroupFilter
from bokeh import events
from bokeh.io import curdoc
from bokeh.models.tools import HoverTool
from bokeh.palettes import Colorblind4
from bokeh.models.glyphs import Text

output_notebook()
output_file("tmc.html", title='Transition Metal Compounds')

Loading BokehJS ...

In [135]:
### Import, prepare data
df = pd.read_csv('../datasets/features_icsd_tmetal-compounds.csv')
df = df[df.heteroanion == False]
df = df[['formula', 'anions', 'heteroanion','sg_sym', 'mm_dist','tm1','oxi1','delta','normed_dist',
        'd_state1', 'n1', 'ionic_r_1', 'mn1', 'gii', 'pred_d1']].dropna()

### Partitioning data
anion_set = ['N','O','S','Se']
anion_names = ['Nitrides', 'Oxides','Sulfides', 'Selenides']
markers = ['x','circle', 'plus', 'triangle']
colors = ['blue', 'red', 'green', 'dimgrey'] #Colorblind4

df = df[np.isin(df.anions, anion_set)]
source = ColumnDataSource(data=dict(x0=df.mm_dist, anions=df.anions, x1=df.normed_dist, y=df.gii, formula=df.formula, 
                                    sg_sym=df.sg_sym, pred_d1=df.pred_d1, delta=df.delta, n1=df.n1, mn1=df.mn1))
# df

In [136]:
### Create plot
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

### Metal-Metal Distance vs GII
left = figure(tools=TOOLS, width=1550, height=850, x_range=(0, 9), y_range=(-0.01, 1), 
           title="Metal-Metal Distance vs. GII")
left.title.align = 'center'

for anion, label, color, marker in zip(anion_set, anion_names, colors, markers):
    view = CDSView(source=source, filters=[GroupFilter(column_name='anions', group=anion)])
    glyph = Text(x="x0", y="y", text="N", text_color=color)
    left.scatter('x0', 'y', source=source, view=view, fill_alpha=0.01, line_alpha=0.5, 
                legend_label=label, color=color, marker=marker, size=9, muted_alpha=0, line_width=1.5)
#     left.add_glyph('x0', 'y', source=source, view=view, fill_alpha=0, line_alpha=0.5, 
#                 legend_label=label, color=color, size=7, muted_alpha=0)
left.xaxis.axis_label = "Metal-Metal Distance (Angstroms)"
left.yaxis.axis_label = "Global Instability Index"
left.xaxis.axis_label_text_font_style = "normal"
left.yaxis.axis_label_text_font_style = "normal"
# Shaded area
left.patch([-9999,-9999,9999,9999],[-0.01,0.2,0.2,-0.01],alpha=0.2, line_width=0, 
        legend_label='Stability Metric', muted_alpha=0)
# Legend
left.legend.background_fill_alpha = 0.8
left.legend.click_policy="mute"
# Fonts
STD_FONT_SIZE = 16
left.title.text_font_size = f'{int(STD_FONT_SIZE*1.2)}pt'
left.xaxis.axis_label_text_font_size = f'{STD_FONT_SIZE}pt'
left.yaxis.axis_label_text_font_size = f'{STD_FONT_SIZE}pt'
left.xaxis.major_label_text_font_size = f'{int(STD_FONT_SIZE*0.8)}pt'
left.yaxis.major_label_text_font_size = f'{int(STD_FONT_SIZE*0.8)}pt'
left.legend.label_text_font_size = f'{STD_FONT_SIZE}pt'

### Delta vs GII
# right = figure(tools=TOOLS, width=400, height=450, x_range=(0.4, 3), y_range=left.y_range, 
#            title="Normalized Metal-Metal Distance vs. GII")
# right.title.align = 'center'
# right.circle('x1', 'y', source=source, color='black', fill_alpha=0, line_alpha=0.2)
# right.xaxis.axis_label = "Metal-Metal Distance / Pure Metal "
# right.yaxis.visible = False
# right.xaxis.axis_label_text_font_style = "normal"
# # Shaded area
# right.patch([-9999,-9999,9999,9999],[-0.01,0.2,0.2,-0.01],alpha=0.2, line_width=0)

### Tooltips
hover = HoverTool()
hover.tooltips ="""
    <div>
        <h3>@formula</h3>
        <div><strong>Spacegroup:    </strong>@sg_sym</div>
        <div><strong>Atomic #:      </strong>@n1</div>
        <div><strong>Pred. d state: </strong>@pred_d1</div>
        <div><strong>Mendeleev #:   </strong>@mn1</div>
    </div>
"""
left.add_tools(hover)
# right.add_tools(hover)

p = gridplot([[left]])#, right]])

### Define the layout and show the interactive
# widgets = column(amp_slider, freq_slider, phase_slider, offset_slider)
show(p)